In [1]:
import jos3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import dataset
df_dataset = pd.read_csv('./dataset/230322_OlderPredictTc_data_thermal.csv')

In [2]:
def simulate_trial(sample: pd.DataFrame, clo_males: float, clo_females: float, PAR: float, Va: float, isCooling: bool) -> pd.DataFrame:
    ids = sample.id_all.unique()
    all_core_temps = []
    all_skin_temps = []
    for i in ids:
        individual = pd.DataFrame(sample[sample.id_all == i])
        # Get parameters
        isFemale = individual.female.iloc[0]
        sex = 'female' if isFemale else 'male'
        clo = clo_females if isFemale else clo_males
        height = individual.height.iloc[0] / 100
        weight = individual.mass.iloc[0]
        age = individual.age.iloc[0]
        fat = individual.bf.iloc[0]
        # Build model (individual)
        model = jos3.JOS3(height=height, weight=weight, age=age, fat=fat, sex=sex)
        model.Icl = clo # Clothing level
        model.PAR = PAR  # Physical activity ratio [-]
        model.posture = 'sitting'  # Whether sitting or standing ??
        # Simulate 60 minutes in air-con before entering room
        model.Ta = 23
        model.RH = 50
        model.simulate(60)
        # Environmental parameters
        model.Ta = individual.ta_set.iloc[0]  # Operative temperature [oC]
        model.RH = individual.rh_set.iloc[0]  # Relative humidity [%]
        model.Va = Va  # Air velocity [m/s]
        # Predict temperature
        if isCooling:
            # Predict temperature
            model.simulate(225)
            # Air con
            model.Ta = 23  # Operative temperature [oC]
            model.RH = 50  # Relative humidity [%]
            model.simulate(120)
            # Back into heat
            model.Ta = individual.ta_set.iloc[0]  # Operative temperature [oC]
            model.RH = individual.rh_set.iloc[0]  # Relative humidity [%]
            model.simulate(individual.shape[0] - 346)
        else:
            model.simulate(individual.shape[0] - 1)  # Exposure time = 540 [min]
        # Extract data
        df_results = pd.DataFrame(model.dict_results())  # Make pandas.DataFrame
        core_temp = df_results.TcrPelvis.tolist()[60:] # drop the first 60 minutes of air-con
        skin_temp = df_results.TskMean.tolist()[60:] # drop the first 60 minutes of air-con
        all_core_temps.append(core_temp)
        all_skin_temps.append(skin_temp)
    # Flatten list before returning
    all_core_temps = [temp for sublist in all_core_temps for temp in sublist]
    all_skin_temps = [temp for sublist in all_skin_temps for temp in sublist]
    return all_core_temps, all_skin_temps

In [3]:
# Get sample
from helpers import get_sample
def run_and_save_trial(study, condition, clo_males, clo_females, PAR, Va, isCooling):
    # Get sample
    sample = get_sample(study, condition)
    # Calculate core temps
    results = simulate_trial(sample, clo_males, clo_females, PAR, Va, isCooling)
    all_core_temps = results[0]
    all_skin_temps = results[1]
    # Check lengths
    print(len(all_core_temps))
    print(sample.shape[0])
    # Save to csv
    df = pd.DataFrame(all_core_temps, columns=["tre_predicted"])
    df["mtsk_predicted"] = all_skin_temps
    df.to_csv('results/takahashi-{}-{}.csv'.format(study, condition), index=False)

In [4]:
# Simulate using JOS-3
# Clo values from "2021 Ashrae handbook - Fundamentals"
# (briefs + shorts + slippers) or (briefs+ shorts + slippers + tshirt)
run_and_save_trial('heatwave 1 (prolonged)', 'hot', clo_males=0.15, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=False)

run_and_save_trial('heatwave 2 (indoor)', 'cool', clo_males=0.23, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'temp', clo_males=0.23, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'warm', clo_males=0.23, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=False)
run_and_save_trial('heatwave 2 (indoor)', 'hot', clo_males=0.23, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=False)

run_and_save_trial('heatwave 3 (cooling)', 'hot', clo_males=0.15, clo_females=0.23, PAR=1.2, Va=0.1, isCooling=True)

31860
31860
7680
7680
7680
7680
17760
17760
16800
16800
10260
10260
